In [ ]:
# Machine Learning - model training
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

import pickle
import sys
sys.path.append('/Users/wolfsinem/product-tagging')

from product_tagging.tags_generator import tokenized_list

In [ ]:
# created dataframe with new tags column 
n = 2000 # number of rows
model_df = tokenized_list()

### Clean data

In [ ]:
model_df.dropna(inplace=True)

In [ ]:
model_df.shape

### Preprocessing

#### MultiLabelBinarizer

In [ ]:
model_df = model_df[:n]

In [ ]:
target_variable = model_df['tags']

In [ ]:
mlb = MultiLabelBinarizer()
target_variable = mlb.fit_transform(target_variable)

In [ ]:
mlb.classes_

#### TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(strip_accents='unicode', 
                             analyzer='word', 
                             ngram_range=(1,3), 
                             stop_words='english',
                             token_pattern=r'\w{3,}'
                            )


# fit the independent features
independent_variable = vectorizer.fit_transform(model_df['description'])

print('Independent variable shape: {}'.format(independent_variable.shape))
print('Target variable shape: {}'.format(target_variable.shape))

In [ ]:
vectorizer.vocabulary_

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                        independent_variable, 
                                        target_variable, 
                                        test_size=0.2, 
                                        random_state=42, 
                                        )

In [ ]:
print('X train shape: {}'.format(X_train.shape))
print('y train shape: {}'.format(y_train.shape))

print('------------------------------')

print('X test shape: {}'.format(X_test.shape))
print('y test shape: {}'.format(y_test.shape))

In [ ]:
Linear_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LinearSVC(
                                                class_weight='balanced',
                                                random_state=42,
                                                tol=1e-1,
                                                C=8.385), 
                                                n_jobs=-1)),
            ])

In [ ]:
Linear_pipeline.fit(X_train, y_train)

In [ ]:
prediction = Linear_pipeline.predict(X_test)
print('Accuracy for LinearSVC is {}'.format(accuracy_score(y_test, prediction)))

In [ ]:
string_1 = ['Key Features of Alisha Solid Womens Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Womens Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Womens Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts']

In [ ]:
model_string_1 = vectorizer.transform(string_1)
Linear_pipeline.predict(model_string_1)

In [ ]:
predicted_tags = mlb.inverse_transform(Linear_pipeline.predict(model_string_1))
predicted_tags

### Save model to deploy

In [ ]:
with open('text_classifier', 'wb') as picklefile:
    pickle.dump(Linear_pipeline,picklefile)

In [ ]:
with open('vect', 'wb') as picklefile:
    pickle.dump(vectorizer,picklefile)

In [ ]:
with open('text_classifier', 'rb') as training_model:
    model = pickle.load(training_model)

In [ ]:
with open('vect', 'rb') as training_model:
    vect = pickle.load(training_model)

In [ ]:
y_pred2 = model.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred2)) 

In [ ]:
string_1 = ['Key Features of Alisha Solid Womens Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Womens Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Womens Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts']

In [ ]:
string_1 = vectorizer.transform(string_1).toarray()

In [ ]:
label = model.predict(string_1)

In [ ]:
mlb.inverse_transform(label)